## 3 Merge and Clean Data

This code reads previously extracted data on county-level drug mortality and socioeconomic data from the US Census Bureau. The data sets are merged and cleaned. The resulting data set, which contains 3135 rows and 51 columns, is saved as a csv file for later use.

## 3.1 Merge Drug Mortality Data with Features / Independent Variables

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in aggregated drug mortality data
drug_mort_counties = pd.read_csv('./data/drug_mort_counties.csv')
drug_mort_counties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3136 entries, 0 to 3135
Data columns (total 6 columns):
FIPS         3136 non-null int64
dm_mean      3136 non-null float64
dm_median    3136 non-null int64
dm_max       3136 non-null int64
dm_min       3136 non-null int64
FIPS.1       3136 non-null int64
dtypes: float64(1), int64(5)
memory usage: 147.1 KB


In [3]:
# read in census data prepared earlier
census = pd.read_csv('./data/county_data_acs5_2015.csv')
census = census.drop(columns = ['Unnamed: 0', 'FIPS'])
census = census.rename(columns = {'FIPS_NCHS' : 'FIPS'})
#census.info()

In [4]:
# merge data
opioid_us = census.merge(drug_mort_counties, on = 'FIPS', how = 'left')
#opioid_us.info()

In [5]:
# Save for later
opioid_us.to_csv('./data/US_drug_mort_census.csv')

## 3.2 Data Cleaning

In [6]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("The dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [7]:
# Finds columns with missing values
missing_values_table(opioid_us)

The dataframe has 75 columns.
There are 8 columns that have missing values.


,Missing Values,% of Total Values
med_hh_inc,1,0.0
gini_index,1,0.0
inc_per_capita,1,0.0
dm_mean,1,0.0
dm_median,1,0.0
dm_max,1,0.0
dm_min,1,0.0
FIPS.1,1,0.0


In [8]:
# For some reason FIPS 46102 (Oglala Lakota County) is missing from 
#   drug_mort_2015 - this is a change of FIPS code
#check_miss = opioid_us[opioid_us.drug_mort.isna()]

opioid_us = opioid_us[opioid_us['dm_mean'].notnull()]

In [9]:
# Finds columns with missing values
missing_values_table(opioid_us)

The dataframe has 75 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
med_hh_inc,1,0.0
gini_index,1,0.0
inc_per_capita,1,0.0


In [10]:
#opioid_us.info()

In [11]:
#Get the row that has med_hh_inc missing
row_pos = opioid_us.index[opioid_us['med_hh_inc'].isnull()][0]
row_pos

2667

In [12]:
# Loving Counti TX has missing data that can be found elsewhere
#gini index:
# http://www.civicdashboards.com/county/loving-county-tx-05000US48301/gini_index
#income data:
#http://www.txcip.org/tac/census/profile.php?FIPS=48301

#Get row of instance that has missing values
row_pos = opioid_us.index[opioid_us['med_hh_inc'].isnull()][0]-1
print(row_pos)

#Get column with missing value and set to correct value
col_pos = opioid_us.columns.get_loc('med_hh_inc')
opioid_us.iat[row_pos, col_pos] = 64179
print(row_pos, col_pos)

print(opioid_us.iat[row_pos, col_pos])

#Get column with missing value and set to correct value
col_pos = opioid_us.columns.get_loc('inc_per_capita')
opioid_us.iat[row_pos, col_pos] = 39221
print(row_pos, col_pos)

print(opioid_us.iat[row_pos, col_pos])

#Get column with missing value and set to correct value
col_pos = opioid_us.columns.get_loc('gini_index')
opioid_us.iat[row_pos, col_pos] = .42
print(row_pos, col_pos)

print(opioid_us.iat[row_pos, col_pos])

2666
2666 58
64179.0
2666 60
39221.0
2666 59
0.42


In [13]:
opioid_us.index[opioid_us['med_hh_inc']==64179]

Int64Index([2667], dtype='int64')

In [14]:
missing_values_table(opioid_us)

The dataframe has 75 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [15]:
# combine age categories to reduce number of fields
opioid_us['age_under_18_p'] = (opioid_us.age_under_5_p + opioid_us.age_5_9_p + opioid_us.age_10_14_p + opioid_us.age_15_17_p + 
                               opioid_us.age_18_19_p + opioid_us.age_15_17_p)
opioid_us['age_18_39_p'] = (opioid_us.age_18_19_p + opioid_us.age_20_p + opioid_us.age_21_p + opioid_us.age_22_24_p + 
                            opioid_us.age_25_29_p + opioid_us.age_30_34_p + opioid_us.age_35_39_p)
opioid_us['age_40_64_p'] = (opioid_us.age_40_44_p + opioid_us.age_45_49_p + opioid_us.age_50_54_p + opioid_us.age_55_59_p + 
                            opioid_us.age_60_61_p + opioid_us.age_62_64_p)
opioid_us['age_64_over_p'] = (opioid_us.age_65_66_p + opioid_us.age_67_69_p + opioid_us.age_70_74_p + opioid_us.age_75_79_p + 
                            opioid_us.age_80_84_p + opioid_us.age_85_over_p)

# combine race categories
opioid_us['race_all_other_p'] = (opioid_us.race_native_amer_p + opioid_us.race_asian_p + 
                                opioid_us.race_pac_islander_p + opioid_us.race_other_p + opioid_us.race_multi_p)

opioid_us_num = opioid_us.drop(columns = ['urban_2006', 
                                        'age_under_5_p', 
                                        'age_5_9_p', 
                                        'age_10_14_p', 
                                        'age_15_17_p', 
                                        'age_18_19_p', 
                                        'age_20_p', 
                                        'age_21_p', 
                                        'age_22_24_p', 
                                        'age_25_29_p', 
                                        'age_30_34_p', 
                                        'age_35_39_p', 
                                        'age_40_44_p', 
                                        'age_45_49_p', 
                                        'age_50_54_p', 
                                        'age_55_59_p', 
                                        'age_60_61_p', 
                                        'age_62_64_p', 
                                        'age_65_66_p', 
                                        'age_67_69_p', 
                                        'age_70_74_p', 
                                        'age_75_79_p', 
                                        'age_80_84_p', 
                                        'age_85_over_p',
                                        'race_native_amer_p',
                                        'race_asian_p', 
                                        'race_pac_islander_p',
                                        'race_other_p', 
                                        'race_multi_p'])

In [16]:
#Save for later use
opioid_us_num.to_csv('./data/US_drug_mort_census_num.csv', index=False)

In [17]:
opioid_us_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3135 entries, 0 to 3135
Data columns (total 51 columns):
FIPS                   3135 non-null int64
totpop                 3135 non-null int64
sex_male_p             3135 non-null float64
sex_female_p           3135 non-null float64
race_white_p           3135 non-null float64
race_black_p           3135 non-null float64
kid_home_fam_p         3135 non-null float64
kid_home_fam2_p        3135 non-null float64
kid_home_fam1_p        3135 non-null float64
kid_home_nonfam_p      3135 non-null float64
adult_home_alone_p     3135 non-null float64
adult_home_fam_p       3135 non-null float64
adult_home_nonfam_p    3135 non-null float64
marry_never_p          3135 non-null float64
married_p              3135 non-null float64
marry_past_p           3135 non-null float64
ed_at_HS_p             3135 non-null float64
ed_at_some_coll_p      3135 non-null float64
ed_at_coll_grad_p      3135 non-null float64
pov_p                  3135 non-null float